In [1]:
import cv2
import os
import numpy as np
import signal
import sys


class DatasetCollector:
    def __init__(self):
        self.expressions = {
            '1': 'angry',
            '2': 'disgust',
            '3': 'fear',
            '4': 'happy',
            '5': 'neutral',
            '6': 'sad',
            '7': 'surprise'
        }
        self.dataset_dir = 'dataset'
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.current_expression = 'neutral'
        self.counters = {expr: 0 for expr in self.expressions.values()}
        self.cap = None
        self.running = True

        # Create dataset directories
        for expression in self.expressions.values():
            os.makedirs(f'{self.dataset_dir}/{expression}', exist_ok=True)
            # Count existing images
            existing_files = os.listdir(f'{self.dataset_dir}/{expression}')
            self.counters[expression] = len([f for f in existing_files if f.endswith(('.jpg', '.png', '.jpeg'))])

        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        """Handle Ctrl+C gracefully"""
        print("\n\n🛑 Received interrupt signal. Shutting down gracefully...")
        self.running = False

    def initialize_camera(self):
        """Initialize camera with error handling"""
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                # Try different camera indices
                for i in range(1, 4):
                    self.cap = cv2.VideoCapture(i)
                    if self.cap.isOpened():
                        print(f"✅ Camera found at index {i}")
                        break
                else:
                    print("❌ No camera found!")
                    return False

            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            self.cap.set(cv2.CAP_PROP_FPS, 30)
            return True

        except Exception as e:
            print(f"❌ Camera initialization error: {e}")
            return False

    def capture_dataset(self):
        """Capture facial expression dataset from webcam"""
        if not self.initialize_camera():
            return

        print("🎭 Facial Expression Dataset Collector")
        print("=" * 50)
        print("\n🎮 CONTROLS:")
        print("   1 - Angry")
        print("   2 - Disgust")
        print("   3 - Fear")
        print("   4 - Happy/Smile")
        print("   5 - Neutral")
        print("   6 - Sad")
        print("   7 - Surprise")
        print("   SPACE - Capture image")
        print("   Q - Quit")
        print("   Ctrl+C - Emergency quit")
        print("\n💡 Make different facial expressions and press SPACE to capture!")
        print(f"📁 Current expression: {self.current_expression}")

        try:
            while self.running:
                ret, frame = self.cap.read()
                if not ret:
                    print("❌ Failed to grab frame")
                    break

                # Convert to grayscale for face detection
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detect faces
                faces = self.face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(100, 100))

                # Draw face bounding box and info
                for (x, y, w, h) in faces:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    # Display expression info
                    info_text = f"Expression: {self.current_expression}"
                    count_text = f"Count: {self.counters[self.current_expression]}"

                    cv2.putText(frame, info_text, (x, y - 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(frame, count_text, (x, y - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                # Display instructions
                cv2.putText(frame, f"Current: {self.current_expression}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
                cv2.putText(frame, "Press 1-7 to change expression", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                cv2.putText(frame, "SPACE to capture, Q to quit", (10, 85),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                cv2.putText(frame, f"Total captured: {sum(self.counters.values())}", (10, 110),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                cv2.imshow('Facial Expression Dataset Collector', frame)

                key = cv2.waitKey(1) & 0xFF

                if key == ord('q'):
                    break
                elif key in [ord(str(i)) for i in range(1, 8)]:
                    # Change expression
                    self.current_expression = self.expressions[chr(key)]
                    print(f"📝 Changed expression to: {self.current_expression}")
                elif key == ord(' '):
                    # Capture image when space is pressed and face is detected
                    if len(faces) > 0:
                        self.save_face_image(gray, faces[0])
                    else:
                        print("❌ No face detected! Please position your face in the frame.")

        except Exception as e:
            print(f"❌ Error during capture: {e}")

        finally:
            self.cleanup()

        print(f"\n✅ Dataset collection complete!")
        print(f"📊 Total images captured: {sum(self.counters.values())}")
        for expr, count in self.counters.items():
            print(f"   {expr}: {count} images")

    def save_face_image(self, gray_frame, face_rect):
        """Save cropped face image"""
        try:
            x, y, w, h = face_rect

            # Expand the face region slightly
            margin = 20
            x = max(0, x - margin)
            y = max(0, y - margin)
            w = min(gray_frame.shape[1] - x, w + 2 * margin)
            h = min(gray_frame.shape[0] - y, h + 2 * margin)

            # Extract and resize face
            face_img = gray_frame[y:y + h, x:x + w]
            face_img = cv2.resize(face_img, (48, 48))

            # Save image
            filename = f"{self.dataset_dir}/{self.current_expression}/{self.current_expression}_{self.counters[self.current_expression]:04d}.jpg"
            success = cv2.imwrite(filename, face_img)

            if success:
                self.counters[self.current_expression] += 1
                print(f"💾 Saved: {filename}")
            else:
                print(f"❌ Failed to save: {filename}")

        except Exception as e:
            print(f"❌ Error saving image: {e}")

    def cleanup(self):
        """Cleanup resources"""
        try:
            if self.cap and self.cap.isOpened():
                self.cap.release()
            cv2.destroyAllWindows()
            # Additional cleanup to ensure windows close
            for i in range(5):
                cv2.waitKey(1)
        except Exception as e:
            print(f"⚠️ Cleanup warning: {e}")


def main():
    print("🚀 Starting Facial Expression Dataset Collector...")
    collector = DatasetCollector()
    collector.capture_dataset()


if __name__ == "__main__":
    main()

🚀 Starting Facial Expression Dataset Collector...
🎭 Facial Expression Dataset Collector

🎮 CONTROLS:
   1 - Angry
   2 - Disgust
   3 - Fear
   4 - Happy/Smile
   5 - Neutral
   6 - Sad
   7 - Surprise
   SPACE - Capture image
   Q - Quit
   Ctrl+C - Emergency quit

💡 Make different facial expressions and press SPACE to capture!
📁 Current expression: neutral
📝 Changed expression to: angry
📝 Changed expression to: fear
📝 Changed expression to: happy
📝 Changed expression to: neutral
📝 Changed expression to: sad
📝 Changed expression to: surprise
📝 Changed expression to: disgust
📝 Changed expression to: angry
📝 Changed expression to: disgust
📝 Changed expression to: fear
📝 Changed expression to: happy
📝 Changed expression to: neutral
📝 Changed expression to: sad
📝 Changed expression to: surprise

✅ Dataset collection complete!
📊 Total images captured: 28709
   angry: 3995 images
   disgust: 436 images
   fear: 4097 images
   happy: 7215 images
   neutral: 4965 images
   sad: 4830 images
  